https://github.com/openai/whisper

In [ ]:
# !pip install openai
# !pip install whisper
# !pip install pytube
# !pip install google-search-results
# !pip install langchain

In [ ]:
import os
import openai
# from langchain.llms import OpenAI


# YouTube の文字起こし

In [ ]:
import whisper
from pytube import YouTube

In [ ]:
url = "https://www.youtube.com/watch?v=h3052XnZhVI"
video =  YouTube(url)


In [ ]:
print(video.streams.get_highest_resolution())
print(video.streams.get_highest_resolution().filesize/1024/1024, 'MB')

## 音声抽出

In [ ]:
audio = video.streams.get_audio_only()
# video.streams.get_highest_resolution().filesize

In [ ]:
video.streaming_data
video.views
video.title
video.register_on_progress_callback
video.rating
video.metadata
video.length
video.description

In [ ]:
os.makedirs('../audio_files/', exist_ok=True)

In [ ]:
audio.download(output_path='../audio_files/')

# Whisper

In [ ]:
# !pip install git+https://github.com/openai/whisper.git -q
# !pip install -U openai-whisper
# !pip install ffmpeg-python
# !brew install ffmpeg # 20分かかった
# !pip install setuptools-rust

In [ ]:
import os
import time
import whisper

## モデルのダウンロードと、audioデータの取り出し

In [ ]:
model_size = 'small' # large medium small base tinyから選択
model = whisper.load_model(model_size) # Install Certificates.commandをダブルクリックして更新

# 開始時間の記録

In [ ]:
start_time = time.time()

# オーディオデータの読み込み

In [ ]:
audio = whisper.load_audio('../audio_files/SoftBank World 2023 孫 正義 特別講演 AGIを中心とした新たな世界へ.mp4')
transcription = model.transcribe('../audio_files/SoftBank World 2023 孫 正義 特別講演 AGIを中心とした新たな世界へ.mp4')
transcription

# 書きこし時間

In [ ]:
print(round(time.time() - start_time), 'sec')

In [ ]:
transcription.keys()

In [ ]:
transcription['segments']

In [ ]:
transcription['segments'][1]['text']

In [ ]:
os.makedirs('../text_files/', exist_ok=True)

In [ ]:
with open(f'../text_files/softbank_world_2023_whisper_{model_size}.txt', 'w') as file:
    # Write each item in the list to the file
    for i in range(len(transcription['segments'])):
        file.write(str(transcription['segments'][i]['text']) + "\n")

In [ ]:
from datetime import datetime
import time

texts = []
start_times = []

for segment in transcription['segments']:
    text = segment['text']
    start = segment['start']
    # print(start)
    start_datetime = datetime.fromtimestamp(start)
    # print(start_datetime)
    formatted_start_time = start_datetime.strftime('00:%M:%S')
    print(formatted_start_time)

    # texts.append("".join(text))
    texts.append(text)
    start_times.append(formatted_start_time)

print(texts[:10])
print(start_times[:10])

In [ ]:
# !pip install openai langchain
# !pip install faiss-cpu

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chains import VectorDBQAWithSourcesChain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import OpenAI
import openai
import faiss

import os


In [ ]:
os.environ['OPENAI_API_KEY'] = "sk-"

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1500, separator = '¥n')
# test_splitter = CharacterTextSplitter(chunk_size=10, separator = 'で')

In [ ]:
text_splitter.split_text(texts[7])

In [ ]:
text_splitter_test = CharacterTextSplitter(
    separator = "_",  # セパレータ
    chunk_size = 12,  # チャンクの文字数
    chunk_overlap = 0,
)


In [ ]:
print(text_splitter_test.split_text('あいうえお_かきくけこ'))

In [ ]:
docs = []
metadatas = []

for i, d in enumerate(texts):
    split = text_splitter.split_text(d)
    docs.extend(split)
    metadatas.extend([{'source':start_times[i]}] * len(split))

embeddings = OpenAIEmbeddings()


In [ ]:
metadatas

In [ ]:
store = FAISS.from_texts(docs, embeddings, metadatas=metadatas)

In [ ]:
store

In [ ]:
faiss.write_index(store.index, 'docs.index')

In [ ]:
# chains = RetrievalQAWithSourcesChain.from_llm(llm=OpenAI(temperature=0), vectorstore=store)
chains = VectorDBQAWithSourcesChain.from_llm(llm=OpenAI(temperature=0), vectorstore=store) #削除よてい

In [ ]:
result = chains({'question': '勉強方法について'})

In [ ]:
result.keys()
result['answer']

In [ ]:
# test
import numpy as np
text2 = 'I have a few pens'
print(np.array(embeddings.embed_query(text2)) .shape)
print(np.array(embeddings.embed_documents(text2)) .shape)